In [1]:
%cd /kaggle/working
%rm -rf /kaggle/working/*

/kaggle/working


In [2]:
import pandas as pd
import yaml
import sys
import os
from glob import glob
import gc
from pathlib import Path
import pickle
from tqdm import tqdm
import numpy as np
from multiprocessing import Pool

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')

MODEL_NAME = "patch_transformer_gru"

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG[MODEL_NAME]["execution"]["best_exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG[MODEL_NAME]['execution']['best_exp_id']}"
os.makedirs(CFG["output_dir"], exist_ok=True)

exp_118


In [3]:
oof = pd.read_parquet(f"{CFG['output_dir']}/oof.parquet")
oof["oof_max"] = oof[["wakeup_oof", "onset_oof"]].max(axis=1)
oof["step"] = oof["step"].astype(int)
oof.head()

,series_id,step,wakeup_oof,onset_oof,oof_max
0,05e1944c3818,6,0.003191,0.000881,0.003191
1,05e1944c3818,18,0.002243,0.000650,0.002243
2,05e1944c3818,30,0.001784,0.000537,0.001784
3,05e1944c3818,42,0.001665,0.000468,0.001665
4,05e1944c3818,54,0.001548,0.000423,0.001548


In [4]:
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
labels.head()

,series_id,night,event,step,timestamp
0,038441c925bb,1,onset,4992.0,2018-08-14T22:26:00-0400
1,038441c925bb,1,wakeup,10932.0,2018-08-15T06:41:00-0400
2,038441c925bb,2,onset,20244.0,2018-08-15T19:37:00-0400
3,038441c925bb,2,wakeup,27492.0,2018-08-16T05:41:00-0400
4,038441c925bb,3,onset,39996.0,2018-08-16T23:03:00-0400


In [6]:
# 次の候補を決定
next_cand_size = 0
count = 0
next_dict = {}
for series_id, train_df in oof.groupby("series_id"):
    train_df = train_df[train_df["oof_max"] >= 0.01]
    label_df = labels[labels["series_id"] == series_id]
    pred_steps = train_df["step"].values
    label_steps = label_df["step"].values

    if len(train_df) == 0:
        continue
    next_cand = np.zeros(pred_steps.max() + 100)
    for pred_step in pred_steps:
        next_cand[int(pred_step - 12): int(pred_step + 12)] = 1
    next_cand_size += np.sum(next_cand)
    next_dict[series_id] = np.where(next_cand)[0]

    for label_step in label_steps:
        if label_step < next_cand.shape[0]:
            count += next_cand[int(label_step)]
    
recall = count / len(labels)
print(f"recall: {recall:.4f}")
print(f"next_cand_size: {next_cand_size}")

with open(f"{CFG['output_dir']}/next_cands.pkl", "wb") as f:
    pickle.dump(next_dict, f)

recall: 0.9870
next_cand_size: 13006680.0
